In [1]:
import os
import sys
import copy
import pickle
import concurrent.futures
import numpy as np
import pandas as pd

import cantera as ct
import rmgpy.chemkin

sys.path.append('/work/westgroup/harris.se/autoscience/autoscience_workflow/workflow/scripts/kinetics')
import kineticfun


import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load the base model
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/sensitivity_analysis/base_rmg24'

base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)
print(f'{len(reaction_list)} reactions and {len(species_list)} species')

1556030143.py:7 <module> INFO Thermo file has default temperature range 300.0 to 1000.0 and 1000.0 to 5000.0


1822 reactions and 110 species


In [3]:
base_delay = 0.14857535

In [4]:
ignition_delay_rxns = base_delay * np.ones(len(reaction_list))
ignition_delay_sp = base_delay * np.ones(len(species_list))

In [11]:
# load individual ignition delay files
results_dir = '/work/westgroup/harris.se/autoscience/autoscience/butane/sensitivity_analysis/base_rmg24/models/reactions'
for i in range(0, len(reaction_list)):
    result_file = os.path.join(results_dir, f'delay_{i:04}.txt')
    if not os.path.exists(result_file):
        continue
    with open(result_file, 'r') as f:
        result = f.read()
        try:
            ignition_delay_rxns[i] = result
        except ValueError:
            continue
            
results_dir = '/work/westgroup/harris.se/autoscience/autoscience/butane/sensitivity_analysis/base_rmg24/models/species'
for i in range(0, len(species_list)):
    result_file = os.path.join(results_dir, f'delay_{i:04}.txt')
    if not os.path.exists(result_file):
        continue
    with open(result_file, 'r') as f:
        result = f.read()
        try:
            ignition_delay_sp[i] = result
        except ValueError:
            continue

rxn_df = pd.DataFrame(ignition_delay_rxns, columns=['delay'])
rxn_df.to_csv(os.path.join(results_dir, 'reaction_delays.csv'))
species_df = pd.DataFrame(ignition_delay_sp, columns=['delay'])
species_df.to_csv(os.path.join(results_dir, 'species_delays.csv'))


# rxn_df = pd.read_csv(os.path.join(results_dir, 'reaction_delays.csv'))
# species_df = pd.read_csv(os.path.join(results_dir, 'species_delays.csv'))
# ignition_delay_rxns = rxn_df['delay'].values
# ignition_delay_sp = species_df['delay'].values

In [13]:
ignition_delay_rxns[288]

0.12905077586632863

In [14]:
np.average(ignition_delay_rxns)

0.14856756683843467

In [9]:
ignition_delay_sp[85]

0.17446256129346216

In [11]:
np.average(ignition_delay_sp)

0.149834571732

In [15]:
# sort the reaction list by difference in flame speed
order_rxn = np.arange(0, len(ignition_delay_rxns))
fs_diff = np.abs(ignition_delay_rxns - base_delay)
most_sensitive_rxn = [x for _, x in sorted(zip(fs_diff, order_rxn))][::-1]


order_sp = np.arange(0, len(ignition_delay_sp))
fs_diff = np.abs(ignition_delay_sp - base_delay)
most_sensitive_sp = [x for _, x in sorted(zip(fs_diff, order_sp))][::-1]

In [23]:
species_list[0].thermo

NASA(polynomials=[NASAPolynomial(coeffs=[3.61263,-0.00100893,2.49898e-06,-1.43375e-09,2.58634e-13,-1051.1,2.65271], Tmin=(100,'K'), Tmax=(1817.05,'K')), NASAPolynomial(coeffs=[2.97593,0.00164137,-7.19704e-07,1.25374e-10,-7.915e-15,-1025.86,5.53741], Tmin=(1817.05,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), comment="""Thermo library: BurkeH2O2""")

In [22]:
dir(species_list[0])

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_molecular_weight',
 '_repr_png_',
 'aug_inchi',
 'calculate_cp0',
 'calculate_cpinf',
 'conformer',
 'contains_surface_site',
 'copy',
 'creation_iteration',
 'energy_transfer_model',
 'explicitly_allowed',
 'fingerprint',
 'from_adjacency_list',
 'from_smiles',
 'generate_aug_inchi',
 'generate_energy_transfer_model',
 'generate_resonance_structures',
 'generate_statmech',
 'generate_transport_data',
 'get_augmented_inchi',
 'get_density_of_states',
 'get_enthalpy',
 'get_entropy',
 'get_free_energy',
 'get_heat_capacity',
 'get_henry_law_constant_data',
 'get_liquid_volumetric_mass_transfer_coefficient_data',
 'get_partition_function',
 'get_

In [27]:
# print the top species
for i in range(0, 50):
    index = most_sensitive_sp[i]
    source = 'library'
    if 'library' not in species_list[index].thermo.comment.lower():
        source = 'GAV'
    print(index, '\t', np.round(ignition_delay_sp[index] - base_delay, 6),'\t', species_list[index], '\t', source)

59 	 0.152401 	 SC4H9O2(186) 	 library
58 	 0.15061 	 PC4H9O2(184) 	 library
73 	 -0.061233 	 C4H8OOH1-3O2(225) 	 library
72 	 -0.055475 	 C4H8OOH2-4(223) 	 library
74 	 -0.053398 	 C4H8OOH2-4O2(229) 	 library
85 	 0.025887 	 CC(CCOO)OO(787) 	 GAV
46 	 -0.018037 	 C3KET21(73) 	 library
5 	 0.011104 	 O2(2) 	 library
32 	 -0.005936 	 C2H5(33) 	 library
21 	 -0.005687 	 CH3(18) 	 library
11 	 -0.00521 	 H2O(8) 	 library
57 	 0.005001 	 SC4H9(183) 	 library
37 	 -0.004698 	 CH3O2H(46) 	 library
4 	 0.004518 	 butane(1) 	 library
56 	 -0.00388 	 PC4H9(182) 	 library
45 	 -0.003723 	 CH3COCH2O2(72) 	 library
17 	 0.003346 	 H(14) 	 library
60 	 0.003181 	 SC4H9O2H(187) 	 library
87 	 0.003009 	 [CH2]C(=O)COO(911) 	 library
50 	 0.00272 	 C3H5-A(94) 	 library
38 	 -0.002513 	 C2H5O2(47) 	 library
13 	 0.002506 	 CH4(10) 	 library
43 	 -0.002469 	 CH3COCH2(70) 	 library
63 	 0.002464 	 C4H71-3(190) 	 library
12 	 -0.001761 	 CH2O(9) 	 library
70 	 -0.001585 	 C4H8OOH1-3(219) 	 library
20 	 -0

In [30]:
# print the top reactions
for i in range(0, 50):

    index = most_sensitive_rxn[i]
#     print(index, '\t', reaction_list[index])
    try:
#         if reaction_list[index].family != 'H_Abstraction' and \
#             reaction_list[index].family != 'Disproportionation' and \
#             reaction_list[index].family != 'intra_H_migration' and \
#             reaction_list[index].family != 'R_Addition_MultipleBond':
#             continue
        print(index, '\t', reaction_list[index], '\t', reaction_list[index].family)
    except AttributeError:
        continue
        print(index, '\t', reaction_list[index], '\t', 'PDEP')

288 	 OH(15) + butane(1) <=> H2O(8) + PC4H9(182) 	 CurranPentane
247 	 OH(15) + butane(1) <=> H2O(8) + SC4H9(183) 	 CurranPentane
279 	 SC4H9O2(186) <=> HO2(16) + C4H8-2(189) 	 CurranPentane
323 	 O2(2) + C4H8OOH1-3(219) <=> C4H8OOH1-3O2(225) 	 CurranPentane
281 	 O2(2) + C4H8OOH2-4(223) <=> C4H8OOH2-4O2(229) 	 CurranPentane
316 	 PC4H9O2(184) <=> HO2(16) + C4H8-1(188) 	 CurranPentane
249 	 HO2(16) + butane(1) <=> H2O2(17) + SC4H9(183) 	 CurranPentane
449 	 C4H8OOH1-3(219) <=> OH(15) + C4H8O1-3(214) 	 CurranPentane
317 	 SC4H9O2(186) <=> HO2(16) + C4H8-1(188) 	 CurranPentane
326 	 C4H8OOH1-3O2(225) <=> C4H71-1,3OOH(238) 	 CurranPentane
290 	 HO2(16) + butane(1) <=> H2O2(17) + PC4H9(182) 	 CurranPentane
282 	 C4H8OOH2-4O2(229) <=> C4H72-2,4OOH(244) 	 CurranPentane
1701 	 SC4H9O2(186) + butane(1) <=> SC4H9(183) + SC4H9O2H(187) 	 CurranPentane
354 	 HO2(16) + CH3CHO(35) <=> H2O2(17) + CH3CO(20) 	 FFCM1(-)
450 	 C4H8OOH2-4(223) <=> OH(15) + C4H8O1-3(214) 	 CurranPentane
306 	 C4H8OOH1-3(21